In [1]:
from tabula import read_pdf
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [2]:
# abrir documento da leitura através de ocr

fichas = pd.read_csv("dados_ocr_fichas.csv")

In [3]:
# transformar df para gdf

fichas['geometry'] = fichas['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(fichas, crs='epsg:4326')

In [4]:
nomes_df = []
num_df = []

# Ler as tabelas presentes no documento pdf

for i in range(1, 7): # número de documentos pretendidos
    nume_df = str(i)
    nome_df = "df_" + str(i)
    nomes_df.append(nome_df)
    num_df.append(nume_df)

for (i, valor) in enumerate(num_df):
    nomes_df[i] = read_pdf("../ficheiros_usados/utilizado_tabelas/"+valor+".pdf",pages="all")

In [5]:
# Encontrar a coluna onde se encontram as carateristicas - aquela que tem o "Uso da Edificação"

nome_coluna = []
indice_coluna = []
for (i, ficha) in enumerate(nomes_df):
    for (x, c) in enumerate(ficha[0].columns):
        for y in ficha[0][c]:
            if y == "Uso da edificação":
                nome_coluna.append(ficha[0].columns[x-1])
                indice_coluna.append(x-1)
             
            elif y == "1 Uso da edificação":
                nome_coluna.append(ficha[0].columns[x])
                indice_coluna.append(x)

In [6]:
# Ver quais as linhas que tem os nan (através da lista indice)
# Retirar essas linhas (através do drop)

for (i, ficha) in enumerate(nomes_df):
    indice = []
    for (x, c) in enumerate(ficha[0][nome_coluna[i]]):
        if pd.isna(c):
            indice.append(x)
    ficha[0] = ficha[0].drop(indice)

In [7]:
# Fazer reset do indice e eliminar o anterior indice

for (i, ficha) in enumerate(nomes_df):
    ficha[0] = ficha[0].reset_index()
    ficha[0] = ficha[0].drop(columns="index")

In [8]:
# Retirar as linhas que estão "partidas" em duas e que não têm informação

for (i, ficha) in enumerate(nomes_df):
    for (x, num) in enumerate(ficha[0]["Unnamed: 0"]): 
        if pd.isna(num):
            ficha[0] = ficha[0].drop([x])
    ficha[0] = ficha[0].drop(columns = ["Unnamed: 0"])

In [9]:
# Queremos ficar apenas com as colunas das carateristicas e dos valores
# Para isso vamos analisar a 1ª linha: se esta for 'str' então é porque essa coluna é para manter.
# Caso não seja 'str' então é para eliminar

# Aqui marca-se o que se quer eliminar, adicionando o índice a uma lista. 

for (i, ficha) in enumerate(nomes_df):
    to_delete = []
    for (x, c) in enumerate(ficha[0].iloc[0].values):
        if type(c) != str:
            to_delete.append(x)

# O pandas.drop quer o nome da coluna, não o índice. Por isso tem de transformar os índices em nomes.

    labels = [ficha[0].columns[x] for x in to_delete]

# Agora é só pedir ao pandas para eliminar as colunas com estes nomes (colunas
# e não linhas, daí o ´axis = 1´). Ao fazer 'inplace = True' ele já faz as
# alterações da 'df_nova' de input. 
   
    ficha[0].drop(labels, axis = 1, inplace = True)

In [10]:
# Labels das classes a que queremos atribuir estes valores

coluna_nova = ["uso_edi", "tip", "n_pisos","n_pisos_ac", "n_pisos_ab", "cota", 
               "altura", "area_terreno", "area_cedencia", "n_fogos", "n_fracoes", 
               "area_total_const", "area_const_anexos", "area_total_imp", "area_bruta_priv", 
               "area_bruta_dep", "estimativa_obra", "area_terreno_lote", 
               "indice_imper", "ind_ocupacao", "ind_utilizacao"]

In [11]:
# Criar uma nova lista com os df de forma a que os titulos e os nomes das carateristicas 
# fiquem iguais em todas --> uniformização


lista_new_df = []

for (i, ficha) in enumerate(nomes_df):
    new_df = pd.DataFrame(columns = ["Caraterísticas", "Valores"])
    new_df["Caraterísticas"] = coluna_nova
    new_df["Valores"] = ficha[0][ficha[0].columns[-1]]

    lista_new_df.append(new_df)

In [12]:
# Colocar os valores que são números todos como float

for (i, ficha) in enumerate(lista_new_df):
  # 1. Trocar pontos por nada
  ficha["Valores"][2:] = ficha["Valores"][2:].replace({'\.':''}, regex = True)
  # 2. Trocar vírgulas por pontos
  ficha["Valores"][2:] = ficha["Valores"][2:].replace({',':'.'}, regex = True)
  # 3. Remover caractéres indesejados
  ficha["Valores"][2:] = ficha["Valores"][2:].replace({'m2|m|€|%|-':''}, regex = True)
  # 4. Remover espaços em branco
  ficha["Valores"][2:] = ficha["Valores"][2:].replace({' ':''}, regex = True)
  # 5. Converter para valores numéricos
  ficha["Valores"][2:] = ficha["Valores"][2:].apply(pd.to_numeric,1)

In [13]:
# Criar um DF com os valores inseridos nas fichas 

juncao_fichas_bd = pd.DataFrame()
for (i, value) in enumerate(lista_new_df):
    juncao_fichas_bd = juncao_fichas_bd.append(value.T.iloc[1], ignore_index = True)

C:\Users\mpate\AppData\Local\Temp\ipykernel_13668\2742452826.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  juncao_fichas_bd = juncao_fichas_bd.append(value.T.iloc[1], ignore_index = True)
C:\Users\mpate\AppData\Local\Temp\ipykernel_13668\2742452826.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  juncao_fichas_bd = juncao_fichas_bd.append(value.T.iloc[1], ignore_index = True)


In [14]:
# Atribuição da nova coluna com a nova terminologia às colunas da tabela

juncao_fichas_bd.columns = coluna_nova

In [15]:
# Devido à errada leitura do Ficheiro, e devido à impossibilidade de verificação de outliers (devido aos poucos dados disponíveis),
# teve-se de realizar um ajuste manual a estes dados.

ficha_2_valores = ["Habitação", "T3", "1", "1", "0", "47.19", "2.7", "2396.00", "0.00", "", "", "314.80", "132.57", "286.50",
"174.10", "140.70", "0", "366.20", "77", "0.12", "0.13"]
juncao_fichas_bd.loc[1,coluna_nova] = ficha_2_valores

In [16]:
# Conjunto de dados finais: juntando os dados das Fichas de Áreas com os dados geográficos 
df_final = pd.merge(fichas, juncao_fichas_bd, left_index=True, right_index=True)

In [17]:
# o indice de impermeabilização é um valor entre 0 e 1 ou entre 0 e 100%. Assim, como há diferentes escalas nas diferentes fichas será tudo uniformizado para %. 
for (i, c) in enumerate(df_final.indice_imper):
    if float(c) < 1:
        df_final["indice_imper"].iloc[i] = c*100

C:\Users\mpate\AppData\Local\Temp\ipykernel_13668\1341815298.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["indice_imper"].iloc[i] = c*100


In [18]:
df_final

,Unnamed: 0,Ficheiro,geometry,address,index_right,Dicofre,Freguesia_x,Concelho,Distrito,TAA,...,area_total_const,area_const_anexos,area_total_imp,area_bruta_priv,area_bruta_dep,estimativa_obra,area_terreno_lote,indice_imper,ind_ocupacao,ind_utilizacao
0,0,1_0,POINT (-22311.57333 101148.32817),"Rua do Depósito, 3750-327, Águeda, Portugal",1719,10121,União das freguesias de Águeda e Borralha,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,...,2018.7,373.9,1304.3,1416.8,601.9,1011475.0,2365.8,28.6,0.35,NaN
1,1,2_0,POINT (-33953.30095 99855.00016),"Rua do Rio, 3750-432, Águeda, Portugal",1687,10109,Fermentelos,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,...,314.80,132.57,286.50,174.10,140.70,0,366.20,77,0.12,0.13
2,2,3_0,POINT (-33359.97024 100326.12185),"Rua Alagoa, 3750-455, Águeda, Portugal",1687,10109,Fermentelos,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,...,179.06,41.35,179.06,128.44,50.62,90568.0,909.0,23.0,0.2,NaN
3,3,4_0,POINT (-27188.05853 102325.32619),"Rua de Moçambique, 3750-301, Águeda, Portugal",1719,10121,União das freguesias de Águeda e Borralha,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,...,218.0,NaN,103.0,194.0,127.0,5000.0,NaN,66.5,0.433,NaN
4,4,5_0,POINT (-26178.02360 106910.35867),"Rua da Cavada, 3750-803, Águeda, Portugal",232,10119,Valongo do Vouga,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,...,334.0,156.0,248.0,168.0,166.0,1000.0,812.0,15.15,0.23,NaN
5,5,6_0,POINT (-26656.91924 109625.28876),"Rua Nossa Senhora da Piedade, Águeda, Portugal",1766,10112,Macinhata do Vouga,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,...,647.6,0.0,391.8,436.4,211.2,NaN,289.7,67.0,0.57,NaN


In [19]:
df_final.to_excel("dados_fichas_areas.xlsx") 